In [1]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders - Logistic Regression (~1h)

## Select features

🎯 Haydi `wait_time` ve `delay_vs_expected` değişkenlerinin çok `iyi/kötü review`lar üzerindeki etkisini inceleyelim.

👉 `orders` training_set’imizi kullanarak iki adet `multivariate logistic regression` çalıştıracağız:
- `logit_one` → `dim_is_one_star` tahmini için  
- `logit_five` → `dim_is_five_star` tahmini için.

 

In [2]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Dataset’inizi import edin:

In [4]:
import sys
from pathlib import Path
project_path = Path("C:/Users/meaki/workintech/sprint-15")
sys.path.append(str(project_path))

In [7]:
from olist.order import Order

orders = Order().get_training_data(with_distance_seller_customer=True)

orders.columns

Index(['order_id', 'wait_time', 'expected_wait_time', 'delay_vs_expected',
       'order_status', 'dim_is_five_star', 'dim_is_one_star', 'review_score',
       'number_of_items', 'number_of_sellers', 'price', 'freight_value',
       'distance_seller_customer'],
      dtype='object')

👉 Kullanmak istediğiniz feature’ları bir listede seçin:

⚠️ Data leakage yaratmadığınızdan emin olun (yani target’tan türetilmiş feature’ları seçmeyin)

💡 `wait_time` ve `delay_vs_expected` değişkenlerinin etkisini anlayabilmek için diğer feature’ların etkisini kontrol etmemiz gerekir, bu yüzden listenize ilgili olabilecek tüm feature’ları dahil edin.

In [11]:
# SENİN KODUN BURAYA
features = [
    "wait_time",
    "delay_vs_expected",
    "number_of_items",
    "number_of_sellers",
    "price",
    "freight_value",
    "distance_seller_customer"
]

🕵🏻 Feature’larınızın `multicollinearity` durumunu `VIF index` kullanarak kontrol edin.

* Çok yüksek olmamalıdır (tercihen < 10), böylece partial regression coefficient’larına ve ilgili `p-values` değerlerine güvenebiliriz.
* Verinizi standardize etmeyi unutmayın!
    * Bir `VIF Analysis`, bir feature’ın diğer feature’lara karşı regresyonunu yaparak hesaplanır...
    * Bu yüzden herhangi bir linear regression çalıştırmadan önce feature’ların `scale etkisini kaldırmak` ve eşit öneme sahip olmalarını sağlamak istersiniz!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardize etme:

In [12]:
# SENİN KODUN BURAYA
import numpy as np
import pandas as pd

X = orders[features]

X_scaled = (X - X.mean()) / X.std()

X_scaled.head()

,wait_time,delay_vs_expected,number_of_items,number_of_sellers,price,freight_value,distance_seller_customer
0,-0.431192,-0.161781,-0.264595,-0.112544,-0.513802,-0.652038,-0.979475
1,0.134174,-0.161781,-0.264595,-0.112544,-0.086640,0.000467,0.429743
2,-0.329907,-0.161781,-0.264595,-0.112544,0.111748,-0.164053,-0.145495
3,0.073540,-0.161781,-0.264595,-0.112544,-0.441525,0.206815,2.054621
4,-1.019535,-0.161781,-0.264595,-0.112544,-0.562388,-0.652038,-0.959115


👉 Olası multicollinearity durumlarını analiz etmek için VIF Analysis’inizi çalıştırın:

In [13]:
# SENİN KODUN BURAYA
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = X_scaled.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_scaled.values, i)
    for i in range(X_scaled.shape[1])
]

vif_data

,feature,VIF
0,wait_time,2.624944
1,delay_vs_expected,2.213500
2,number_of_items,1.371316
3,number_of_sellers,1.093349
4,price,1.208582
5,freight_value,1.673727
6,distance_seller_customer,1.442040


## Logistic Regressions

👉 İki adet `Logistic Regression` modeli fit edin:
- `logit_one` → `dim_is_one_star` tahmini için
- `logit_five` → `dim_is_five_star` tahmini için.

`Logit 1️⃣`

In [14]:
# SENİN KODUN BURAYA
import statsmodels.api as sm
X = sm.add_constant(X_scaled)
y_one = orders["dim_is_one_star"]
logit_one = sm.Logit(y_one, X).fit()
logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.273582
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Mon, 23 Feb 2026   Pseudo R-squ.:                  0.1448
Time:                        15:38:07   Log-Likelihood:                -26229.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       -2.4676      0.013   -190.852      0.000      -2.493      -2.442
wait_time                    0.7124      0.017     42.173      0.000       0.679       0.746
delay_vs_expected            0.2553      0.019     13.801      0.000       0.219       0.292
number_of_items              0.2500      0.011     23.634      0.000       0.229       0.271
number_of_sellers            0.1778      0.008     22.642      0.000       0.162       0.193
price                        0.0505      0.011      4.464      0.000       0.028       0.073
freight_value               -0.0219      0.013     -1.697      0.090      -0.047       0.003
distance_seller_customer    -0.1785      0.014    -12.934      0.000      -0.206      -0.151
============================================================================================
"""

`Logit 5️⃣`

In [15]:
# SENİN KODUN BURAYA
y_five = orders["dim_is_five_star"]
logit_five = sm.Logit(y_five, X).fit()
logit_five.summary()

Optimization terminated successfully.
         Current function value: 0.636830
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Mon, 23 Feb 2026   Pseudo R-squ.:                 0.05806
Time:                        15:38:30   Log-Likelihood:                -61054.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        0.3388      0.007     47.340      0.000       0.325       0.353
wait_time                   -0.5219      0.012    -44.693      0.000      -0.545      -0.499
delay_vs_expected           -0.4330      0.024    -18.414      0.000      -0.479      -0.387
number_of_items             -0.1365      0.008    -16.331      0.000      -0.153      -0.120
number_of_sellers           -0.1427      0.008    -18.219      0.000      -0.158      -0.127
price                        0.0208      0.008      2.721      0.007       0.006       0.036
freight_value                0.0053      0.009      0.590      0.555      -0.012       0.023
distance_seller_customer     0.0872      0.008     10.509      0.000       0.071       0.103
============================================================================================
"""

💡 Şimdi bu iki logistic regression’ın sonuçlarını analiz etme zamanı:

- Partial coefficient’ları kendi kelimelerinizle yorumlayın.
- `p-values` kullanarak istatistiksel anlamlılıklarını kontrol edin.
- Coefficient önemleri açısından `logit_one` ve `logit_five` arasında herhangi bir fark görüyor musunuz?

In [18]:
# Aşağıdaki cümlelerden doğru olanları aşağıdaki listeye kaydedin.

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more than one_star"

your_answer = [
    "delay_vs_expected influences five_star ratings even more than one_star ratings"
]

🧪 __Kodunu Test Et__

In [19]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts =============================
platform win32 -- Python 3.11.7, pytest-9.0.2, pluggy-1.6.0 -- C:\Users\meaki\AppData\Local\Programs\Python\Python311\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\meaki\AppData\Local\GitHubDesktop\app-3.5.4\data-logit\tests
plugins: anyio-4.12.0
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.02s ==============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Açıklamalar ve ileri seviye kavramlar</i> -</summary>


> _Diğer tüm şeyler sabitken, `delay factor`, 1-yıldız review alma ihtimalini etkilemesinden bile daha fazla, 5-yıldızdan mahrum kalma ihtimalini artırma eğilimindedir. Muhtemelen bunun sebebi, 1-yıldız review’ların bizzat çok kötü ürünleri hedeflemesi, kötü teslimatları değil._

❗️ Ancak tamamen titiz olmak için, **iki farklı modelin coefficient’larını karşılaştırırken daha dikkatli olmamız gerekir**, çünkü **benzer popülasyonlara dayanmayabilirler**!
    Burada 2 alt popülasyonumuz var: (1-yıldız verenler ve 5-yıldız verenler) ve bunlar doğaları gereği farklı davranış kalıpları sergileyebilirler. 5-yıldız vermeye daha meyilli “mutlu insanlar”ın, “gecikme” veya “fiyat” söz konusu olduğunda, 1-yıldızı “Lucky-Luke gibi ateşleyen” “huysuz insanlara” göre daha az hassas olmaları gayet mümkün...

</details>



🏁 Tebrikler!

💾 `logit.ipynb` notebook’unuzu commit ve push etmeyi unutmayın!